In [2]:
# import block
import cv2
import numpy as np
from pathlib import Path
import os
import csv


# Similarity transformation 
## Working principle of the method

The OpenCV library provides a way to calculate a similarity transformation matrix, which is a 2x3 affine matrix with four degrees of freedom: scale, rotation, x translation, and y translation. This matrix can be used to align images taken in any given channel with a template image (which is by default in channel 4).

Once the similarity transformation matrix is calculated, it can be applied to the image of the respective channel. This ensures that all images are perfectly aligned with channel 4. The process is fast, with the initial calibration step taking around 6 seconds, and applying the matrix transformation for one channel taking around 19 milliseconds. If you apply the transformation for all channels at once, it takes around 60 milliseconds.

Because the process is quick, you can potentially conduct a full "calibration" every time the camera software is initiated.

## Question
And here my first question arises: how do you apply the configuration from the corrections.xml file? Would it be possible (or easier) to just apply the linear transformation for each channel, rather than seeking exact pixel, scale, and rotation angle volumes?

If it's not possible (or not easier), it is possible to get (shift_x, shift_y) values from the matrix by decomposition. The similarity matrix can be represented as:

\begin{bmatrix} \cos(\theta) \cdot s & -\sin(\theta) \cdot s & t_x \\ \sin(\theta) \cdot s & \cos(\theta) \cdot s & t_y \end{bmatrix}

where t_x and t_y are translation (shift) values in pixels, s is the scale, and \theta is the rotation angle.

## Issue
While the scale factor and rotation angle fit perfectly well between values I get by decomposition and  your corrections.xml file, the shift values are always off. I am confident that the similarity matrix is calculated correctly because when I decompose it into separate scale, rotation, and translation matrices and then apply them consecutively to the image, I get the image perfectly aligned with channel 4. This brings us back to the question of how to the configuration values are applied from the corrections.xml file (in which order). In this particular case, the order of transformations is important. The scale should be applied first, then the rotation, and finally the translation. If the order is changed, you will get different results, which could be the reason why x and y shift values don't match between matrix decomposition and values in corrections.xml.

You can see how this works in the code below. Input images are taken from the camera, but you can use any images you like.

In [23]:

def channels_similarity_transform(root_dir=('./Input_images/'), template_img=0):
    ''' This function takes a directory with images and a number of template image 
    and returns a list of similarity matrices for each channel respectively to the template image'''
    # Set the directory
    root_dir=Path(root_dir)
    # Read the images into a list
    images = [cv2.imread(str(root_dir / f)) for f in os.listdir(root_dir)]
    # Convert the images to grayscale
    gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    # Register all images to the first image
    template = gray_images[template_img]

    height, width = template.shape

    # Create ORB detector with 5000 features.
    orb_detector = cv2.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not required in this case).
    # here just for template image
    k_temp, d_temp = orb_detector.detectAndCompute(template, None)
    # initiating a list of all SG matrices
    SG_arr = []
    # iterating through the gray images.
    for num, image in enumerate(gray_images):
        k_ch, d_ch = orb_detector.detectAndCompute(image, None)

        # Match features between the two images.
        # We create a Brute Force matcher with
        # Hamming distance as measurement mode.
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        # Match the two sets of descriptors.
        matches = list(matcher.match(d_temp, d_ch))

        # Sort matches on the basis of their Hamming distance.
        matches.sort(key=lambda x: x.distance)

        # Take the top 90 % matches forward.
        matches = matches[:int(len(matches)*0.9)]
        no_of_matches = len(matches)

        # Define empty matrices of shape no_of_matches * 2.
        pt = np.zeros((no_of_matches, 2))
        p_ch = np.zeros((no_of_matches, 2))

        for i in range(len(matches)):
            pt[i, :] = k_temp[matches[i].queryIdx].pt
            p_ch[i, :] = k_ch[matches[i].trainIdx].pt

        # Find the SG matrix.
        affine_transform, mask = cv2.estimateAffinePartial2D(p_ch, pt, cv2.RANSAC) 
        SG_arr.append(affine_transform)
        # Use this matrix to transform the
        # colored image wrt the reference image.
        transformed_img = cv2.warpAffine(images[num], affine_transform, (width, height))

        # Save the output.
        if os.path.exists("./outputs"):
            pass
        else:
            os.mkdir("./outputs")
        cv2.imwrite(f'./outputs/output_channel{num}.png', transformed_img)
    # now we save homography_arr as csv file

    b = open('SG_matrices.csv', 'w')
    a = csv.writer(b)
    a.writerows(SG_arr)
    b.close()
    return SG_arr


In [24]:
%%time
matrices_arr = channels_similarity_transform(root_dir='z:\\User\\Kashkan\\Multispec\\Dural\\Separated_channels_dural_closeup',template_img=4)
# adjusted files are saved into the outputs folder, 
# Similarity matrices are saved into the atrices_arr variable and the csv file

AttributeError: module 'cv2' has no attribute 'estimateRigidTransform'

## Similarity matrix decomposition
Ideal solution would be to transpose every channel image by the given similarity matrix. The operation itself is quick and can be conducted on the go:

In [44]:
# prep steps (made once during the initial calibration)
img_1 = cv2.imread('./Input_images/1st alignment_ch0.png')
matrices_arr = channels_similarity_transform(template_img=4)
height, width, chan = img_1.shape

In [43]:
%%time
# transformation step (made for each channel)
transformed_img = cv2.warpAffine(1, matrices_arr[1], (width, height))

CPU times: user 104 ms, sys: 27 ms, total: 131 ms
Wall time: 21.1 ms


## Similarity matrix decomposition
However, if not possible (or not desired) we can decompose similarity matrix to Scale factor, rotation angle and translation values from the formula:

\begin{bmatrix} \cos(\theta) \cdot s & -\sin(\theta) \cdot s & t_x \\ \sin(\theta) \cdot s & \cos(\theta) \cdot s & t_y \end{bmatrix}

In [19]:
def similarity_decompose(list_of_arrs):
    ''' This function decomposes the similarity matrix into scale coeficient, rotation degree and pixel shift'''
    for num, array in enumerate(list_of_arrs):
        tn_teta = round(
            round(array[1, 0], 6)/round(array[0, 0], 6), 6)
        teta_degrees = round(np.arctan(tn_teta)*180/np.pi, 6)
        scaling = round(array[0, 0]/np.cos(teta_degrees*np.pi/180), 6)
        print(f"------------channel {num}-----------------")
        print(f"Pixel shift for channel {num} in x is: {round(array[0,2],2)}")
        print(f"Pixel shift for channel {num} in y is: {round(array[1,2],2)}")
        print(f"Rotation for channel {num} is {round(teta_degrees, 4)} degrees: ")
        print(f"Scale coeficient for channel {num} is {scaling}")
        print(f"")

While scale and rotation correspond perfectly well, x and y shifts fo not fit with your corrections.xml. That's why I wonder in what order you apply the transformations?

In [22]:
# Separated_channels_dural_closeup
similarity_decompose(matrices_arr)

------------channel 0-----------------
Pixel shift for channel 0 in x is: 10.22
Pixel shift for channel 0 in y is: -9.21
Rotation for channel 0 is 0.591 degrees: 
Scale coeficient for channel 0 is 0.997827

------------channel 1-----------------
Pixel shift for channel 1 in x is: 6.53
Pixel shift for channel 1 in y is: 2.21
Rotation for channel 1 is 0.1231 degrees: 
Scale coeficient for channel 1 is 0.995381

------------channel 2-----------------
Pixel shift for channel 2 in x is: -11.94
Pixel shift for channel 2 in y is: 14.83
Rotation for channel 2 is -0.6407 degrees: 
Scale coeficient for channel 2 is 1.001186

------------channel 3-----------------
Pixel shift for channel 3 in x is: 6.52
Pixel shift for channel 3 in y is: -8.73
Rotation for channel 3 is 0.4741 degrees: 
Scale coeficient for channel 3 is 1.00278

------------channel 4-----------------
Pixel shift for channel 4 in x is: 0.0
Pixel shift for channel 4 in y is: 0.0
Rotation for channel 4 is 0.0 degrees: 
Scale coeficie

In [20]:
# closeup_rot_scale_shift
similarity_decompose(matrices_arr)

------------channel 0-----------------
Pixel shift for channel 0 in x is: -2.69
Pixel shift for channel 0 in y is: 0.64
Rotation for channel 0 is -0.0123 degrees: 
Scale coeficient for channel 0 is 0.999717

------------channel 1-----------------
Pixel shift for channel 1 in x is: -2.24
Pixel shift for channel 1 in y is: -1.54
Rotation for channel 1 is 0.0154 degrees: 
Scale coeficient for channel 1 is 1.000815

------------channel 2-----------------
Pixel shift for channel 2 in x is: -1.07
Pixel shift for channel 2 in y is: 2.23
Rotation for channel 2 is -0.0171 degrees: 
Scale coeficient for channel 2 is 1.000333

------------channel 3-----------------
Pixel shift for channel 3 in x is: 2.5
Pixel shift for channel 3 in y is: 3.59
Rotation for channel 3 is 0.0163 degrees: 
Scale coeficient for channel 3 is 0.999279

------------channel 4-----------------
Pixel shift for channel 4 in x is: 0.0
Pixel shift for channel 4 in y is: 0.0
Rotation for channel 4 is 0.0 degrees: 
Scale coeficie

In [40]:

def CST(root_dir=('./Input_images/'), template_img=0):
    ''' This function takes a directory with images and a number of template image 
    and returns a list of similarity matrices for each channel respectively to the template image'''
    # Set the directory
    root_dir=Path(root_dir)
    # Read the images into a list
    images = [cv2.imread(str(root_dir / f)) for f in os.listdir(root_dir)]
    # Convert the images to grayscale
    gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    # Register all images to the first image
    template = gray_images[template_img]

    height, width = template.shape

    # Create ORB detector with 5000 features.
    orb_detector = cv2.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not required in this case).
    # here just for template image
    k_temp, d_temp = orb_detector.detectAndCompute(template, None)
    # initiating a list of all SG matrices
    SG_arr = []
    # iterating through the gray images.
    for num, image in enumerate(gray_images):
        k_ch, d_ch = orb_detector.detectAndCompute(image, None)

        # Match features between the two images.
        # We create a Brute Force matcher with
        # Hamming distance as measurement mode.
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        # Match the two sets of descriptors.
        matches = list(matcher.match(d_temp, d_ch))

        # Sort matches on the basis of their Hamming distance.
        matches.sort(key=lambda x: x.distance)

        # Take the top 90 % matches forward.
        matches = matches[:int(len(matches)*0.9)]
        no_of_matches = len(matches)

        # Define empty matrices of shape no_of_matches * 2.
        pt = np.zeros((no_of_matches, 2))
        p_ch = np.zeros((no_of_matches, 2))

        for i in range(len(matches)):
            pt[i, :] = k_temp[matches[i].queryIdx].pt
            p_ch[i, :] = k_ch[matches[i].trainIdx].pt

        # Find the SG matrix.
        affine_transform, mask = cv2.estimateAffinePartial2D(p_ch, pt, cv2.RANSAC)
        SG_arr.append(affine_transform)
        # Use this matrix to transform the
        # colored image wrt the reference image.
        scale_rot_matrix = affine_transform
        scale_rot_matrix.T[2] = [0,0]
        intermed_img = cv2.warpAffine(image, scale_rot_matrix, (width, height))
        # Estimate the x and y shifts.
        dx, dy = cv2.phaseCorrelate(intermed_img.astype(np.float32), template.astype(np.float32))
        print(dx, dy)
        SG_arr.append([scale_rot_matrix[:2, :2], dx, dy])

        # # Save the output.
        # if os.path.exists("./outputs"):
        #     pass
        # else:
        #     os.mkdir("./outputs")
        # cv2.imwrite(f'./outputs/output_channel{num}.png', transformed_img)
    # now we save homography_arr as csv file

    # b = open('SG_matrices.csv', 'w')
    # a = csv.writer(b)
    # a.writerows(SG_arr)
    # b.close()
    # return SG_arr


    for num, array in enumerate(SG_arr):
        print(array)
        tn_teta = round(
            round(array[0][1, 0], 6)/round(array[0][0, 0], 6), 6)
        teta_degrees = round(np.arctan(tn_teta)*180/np.pi, 6)
        scaling = round(array[0][0, 0]/np.cos(teta_degrees*np.pi/180), 6)
        print(f"------------channel {num}-----------------")
        print(f"Pixel shift for channel {num} in x is: {round(array[1],2)}")
        print(f"Pixel shift for channel {num} in y is: {round(array[2],2)}")
        print(f"Rotation for channel {num} is {round(teta_degrees, 4)} degrees: ")
        print(f"Scale coeficient for channel {num} is {scaling}")
        print(f"")

In [31]:
a = np.array([[1,2,3],[1,2,3]])
a.T[2]=[0,0]
a

array([[1, 2, 0],
       [1, 2, 0]])

In [41]:
%%time
matrices_arr = CST(root_dir='z:\\User\\Kashkan\\Multispec\\Dural\\Separated_channels_dural_closeup',template_img=4)
# adjusted files are saved into the outputs folder, 
# Similarity matrices are saved into the atrices_arr variable and the csv file

(10.63190121993921, -9.18973353208753) 0.9868534647623698
(6.266118589790267, 2.5958274559997108) 0.8322906524658203
(-12.09964533221114, 14.884369647757012) 0.7550543505859375
(6.274195081845619, -8.698557770997922) 0.9557598966471355
(0.0, 0.0) 1.0
(-8.799504884518683, 7.865331056747095) 0.9666954747517904
[[ 0.99777372 -0.01029151  0.        ]
 [ 0.01029151  0.99777372  0.        ]]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# Some other methods were checked below
All working, but homography and classic 6DOF affine transformations, but they seems like excessive, in this particular case.

## Homography matrix (2 images)

In [4]:

# Open the image files.
# Image to be aligned.
img1_color = cv2.imread(
    "/mnt/buf/PSI/Multispec/2023.04.21_alignment_photo/Original images/1st alignment_ch0.png")
# Reference image.
img2_color = cv2.imread(
    "/mnt/buf/PSI/Multispec/2023.04.21_alignment_photo/Original images/1st alignment_ch5.png")

# Convert to grayscale.
img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
height, width = img2.shape

# Create ORB detector with 5000 features.
orb_detector = cv2.ORB_create(5000)

# Find keypoints and descriptors.
# The first arg is the image, second arg is the mask
#  (which is not required in this case).
kp1, d1 = orb_detector.detectAndCompute(img1, None)
kp2, d2 = orb_detector.detectAndCompute(img2, None)

# Match features between the two images.
# We create a Brute Force matcher with
# Hamming distance as measurement mode.
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match the two sets of descriptors.
matches = list(matcher.match(d1, d2))

# Sort matches on the basis of their Hamming distance.
matches.sort(key=lambda x: x.distance)

# Take the top 90 % matches forward.
matches = matches[:int(len(matches)*0.9)]
no_of_matches = len(matches)

# Define empty matrices of shape no_of_matches * 2.
p1 = np.zeros((no_of_matches, 2))
p2 = np.zeros((no_of_matches, 2))

for i in range(len(matches)):
    p1[i, :] = kp1[matches[i].queryIdx].pt
    p2[i, :] = kp2[matches[i].trainIdx].pt

# Find the homography matrix.
homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)

% % time
# Use this matrix to transform the
# colored image wrt the reference image.
transformed_img = cv2.warpPerspective(img1_color,
                                      homography, (width, height))

# Save the output.
cv2.imwrite('output.png', transformed_img)


True

## All images in the folder (homography)

In [8]:

def retrieve_images(root_dir=Path('/mnt/buf/PSI/Multispec/Code/Registration(Medium)/FIRE/Images'), template_img=0):

    # Get the list of image filenames (I left only those, which apparently represented same eye)
    image_filenames = [f for f in os.listdir(root_dir)]

    # Read the images into a list
    images = [cv2.imread(str(root_dir / f)) for f in image_filenames]
    # Convert the images to grayscale
    gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    # Register all images to the first image
    template = gray_images[template_img]

    height, width = template.shape

    # Create ORB detector with 5000 features.
    orb_detector = cv2.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not required in this case).
    kt, dt = orb_detector.detectAndCompute(template, None)
    homography_arr = []
    for num, image in enumerate(gray_images):
        k2, d2 = orb_detector.detectAndCompute(image, None)

        # Match features between the two images.
        # We create a Brute Force matcher with
        # Hamming distance as measurement mode.
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        # Match the two sets of descriptors.
        matches = list(matcher.match(dt, d2))

        # Sort matches on the basis of their Hamming distance.
        matches.sort(key=lambda x: x.distance)

        # Take the top 90 % matches forward.
        matches = matches[:int(len(matches)*0.9)]
        no_of_matches = len(matches)

        # Define empty matrices of shape no_of_matches * 2.
        pt = np.zeros((no_of_matches, 2))
        p2 = np.zeros((no_of_matches, 2))

        for i in range(len(matches)):
            pt[i, :] = kt[matches[i].queryIdx].pt
            p2[i, :] = k2[matches[i].trainIdx].pt

        # Find the homography matrix.
        homography, mask = cv2.findHomography(p2, pt, cv2.RANSAC)
        homography_arr.append(homography)
        # Use this matrix to transform the
        # colored image wrt the reference image.
        transformed_img = cv2.warpPerspective(images[num],
                                              homography, (width, height))

        # Save the output.
        cv2.imwrite(f'output_channel{num}.png', transformed_img)
    # now we save homography_arr as csv file

    b = open('homography_matrices.csv', 'w')
    a = csv.writer(b)
    a.writerows(homography_arr)
    b.close()
    b
    return homography_arr


## Homography matrix decomposition (so far failing)

In [9]:
def decompose_homography(H):
    # Extract rotation and translation
    R = H[:2, :2]
    t = H[:2, 2]

    # Calculate rotation angle in degrees
    theta = np.arctan2(R[1, 0], R[0, 0]) * 180 / np.pi

    # Calculate pixel shift in x and y directions
    shift_x = t[0]
    shift_y = t[1]
    # Calculate scaling factor
    s = np.linalg.norm(R[:, 0]) + np.linalg.norm(R[:, 1]) / 2

    return shift_x, shift_y, theta, s


In [10]:
%% time
hom_arr = retrieve_images(template_img=4)
for num, arr in enumerate(hom_arr):
    # Example usage
    shift_x, shift_y, theta, s = decompose_homography(arr)
    print(f"--------------Channel {num}---------------")
    print(f"Pixel shift in x of channel {num} is: {round(shift_x,2)}")
    print(f"Pixel shift in y of channel {num} is: {round(shift_y, 2)}")
    print(f"Rotation in channel {num} is {round(theta, 2)} degrees: ")
    print(f" ")


UsageError: Cell magic `%%` not found.


## Affine transformation variant
(should be a bit faster, but little bit less precise)

In [26]:
# Open the image files.
# Image to be aligned.
img1_color = cv2.imread(
    "/mnt/buf/PSI/Multispec/2023.04.21_alignment_photo/Original images/1st alignment_ch0.png")
# Reference image.
img2_color = cv2.imread(
    "/mnt/buf/PSI/Multispec/2023.04.21_alignment_photo/Original images/1st alignment_ch5.png")

# Convert to grayscale.
img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
height, width = img2.shape

# Create ORB detector with 5000 features.
orb_detector = cv2.ORB_create(5000)

# Find keypoints and descriptors.
kp1, d1 = orb_detector.detectAndCompute(img1, None)
kp2, d2 = orb_detector.detectAndCompute(img2, None)

# Match features between the two images.
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = list(matcher.match(d1, d2))

# Sort matches based on distance.
matches.sort(key=lambda x: x.distance)

# Take the top 90% matches.
matches = matches[:int(len(matches)*0.9)]

# Extract matched keypoints.
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# Find affine transform.
affine, mask = cv2.estimateAffine2D(
    src_pts, dst_pts, method=cv2.RANSAC, ransacReprojThreshold=5)

# Apply transform to the image.
transformed_img = cv2.warpAffine(img1_color, affine, (width, height))

# Save the output.
cv2.imwrite('Affine_output.png', transformed_img)


True

# Geometric similar transformation 
### (affine with 4 DOF - translation (x; y), rotation, scaling)
**Less precise, but easy for decomposition**

In [2]:

# Open the image files.
# Image to be aligned.
img1_color = cv2.imread(
    "/mnt/buf/PSI/Multispec/2023.04.21_alignment_photo/Original images/1st alignment_ch0.png")
# Reference image.
img2_color = cv2.imread(
    "/mnt/buf/PSI/Multispec/2023.04.21_alignment_photo/Original images/1st alignment_ch4.png")

# Convert to grayscale.
img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
height, width = img2.shape

# Create ORB detector with 5000 features.
orb_detector = cv2.ORB_create(5000)

# Find keypoints and descriptors.
# The first arg is the image, second arg is the mask
#  (which is not required in this case).
kp1, d1 = orb_detector.detectAndCompute(img1, None)
kp2, d2 = orb_detector.detectAndCompute(img2, None)

# Match features between the two images.
# We create a Brute Force matcher with
# Hamming distance as measurement mode.
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match the two sets of descriptors.
matches = list(matcher.match(d1, d2))

# Sort matches on the basis of their Hamming distance.
matches.sort(key=lambda x: x.distance)

# Take the top 90 % matches forward.
matches = matches[:int(len(matches)*0.9)]
no_of_matches = len(matches)

# Define empty matrices of shape no_of_matches * 2.
p1 = np.zeros((no_of_matches, 2))
p2 = np.zeros((no_of_matches, 2))

for i in range(len(matches)):
    p1[i, :] = kp1[matches[i].queryIdx].pt
    p2[i, :] = kp2[matches[i].trainIdx].pt

# Find the affine transformation matrix.
affine_transform, mask = cv2.estimateAffinePartial2D(p1, p2, cv2.RANSAC)

# Use this matrix to transform the
# colored image wrt the reference image.
transformed_img = cv2.warpAffine(img1_color, affine_transform, (width, height))

# Save the output.
# cv2.imwrite('output.png', transformed_img)
# cv2.imwrite('img1_orig.png', img1)
# cv2.imwrite('img2_orig.png', img2)


## Affine transform decomposition SVD

In [3]:
import numpy as np

# Extract the translation component
translation = affine_transform[:, 2]

# Extract the rotation and scaling component
rotation_scaling = affine_transform[:, :2]

# Compute the scaling component
u, s, vh = np.linalg.svd(rotation_scaling)
scaling = np.diag(s) @ vh

# Print the results
print("Translation: ", translation)
print("Rotation: ", rotation_scaling)
print("Scaling: ", scaling)


Translation:  [-2.4822889   7.75477429]
Rotation:  [[ 0.99844567  0.00460057]
 [-0.00460057  0.99844567]]
Scaling:  [[ 0.          0.99845627]
 [-0.99845627  0.        ]]


# Conclusion:
Ask Tom how the adjustment happens in the configuration - first should be scale, then rotation, then translation, not vice versa!

In [13]:
affine_transform_scale_rot = np.append(affine_transform.T[:2].T, np.array([[0, 0]]).T, axis=1)
affine_transform_translation = np.append(np.diag((1, 1)), affine_transform[:,2].reshape((-1, 1)), axis=1)

array([[ 0.99844567,  0.00460057,  0.        ],
       [-0.00460057,  0.99844567,  0.        ]])

In [78]:
transformed_img_trans = cv2.warpAffine(img1_color, affine_transform_translation, (width, height))
transformed_img_rot = cv2.warpAffine(transformed_img_trans, affine_transform_scale_rot, (width, height))


# Save the output.
cv2.imwrite('output_trans_rot.png', transformed_img_trans)
cv2.imwrite('img1_orig.png', img1)
cv2.imwrite('img2_orig.png', img2)

True

In [2]:
def SG_decompose(list_of_arrs):
    for num, array in enumerate(list_of_arrs):
        tn_teta = round(
            round(array[1, 0], 6)/round(array[0, 0], 6), 6)
        teta_degrees = round(np.arctan(tn_teta)*180/np.pi, 6)
        scaling = round(array[0, 0]/np.cos(teta_degrees*np.pi/180), 6)
        print(f"Pixel shift for channel {num} in x is: {round(affine_transform[0,2],2)}")
        print(f"Pixel shift for channel {num} in y is: {round(affine_transform[1,2],2)}")
        print(f"Rotation for channel {num} is {round(teta_degrees, 4)} degrees: ")
        print(f"Scale coeficient for channel {num} is {scaling}")
    